# DeBERTa-v3 

### Import necessary packages

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.transformers import log_model
import logging 
from mlflow.sklearn import save_model

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from mlflow.models.signature import infer_signature
from sklearn.utils.class_weight import compute_class_weight


from torch import nn

import sentencepiece
import os

# os.environ["TOKENIZERS_PARALLELISM"] = "false"  # This tells Hugging Face: “Don’t use parallel tokenization — avoid possible deadlocks.”

from torch.utils.data import Dataset, DataLoader
import torch

from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoModel, AutoTokenizer, AutoConfig

import config 

### MLFlow setup


In [21]:
MODEL_NAME = "deberta_v3" 
TRACKING_URI = open("../.mlflow_uri").read().strip()
EXPERIMENT_NAME = config.EXPERIMENT_NAME

logging.basicConfig(format="%(asctime)s: %(message)s") # Configure logging format to show timestamp before every message

logger = logging.getLogger()
logger.setLevel(logging.INFO) # Only show logs that are INFO or more important (e.g., WARNING, ERROR) — but ignore DEBUG.

### Get data

In [22]:
DATA_PATH = "../data/data_small.csv"
MODEL_PATH = "microsoft/deberta-v3-base"
MODEL_TRAINING_PATH ="microsoft/deberta-v3-small"

In [28]:
def tokenize(texts):
    return tokenizer(
        texts,
        padding="max_length", #ensures that all tokenized sequences are padded to the same length, padding adds special tokens to shorter sequeces so they match the maximum length
        truncation=True, #if sequence exceeds max, it will be trucated
        max_length=512, #for most transformer models, 512 is a common limit for maximum length
        return_tensors="pt" #converts the output to pytorch tensors
    )

#object oriented programming (class is the object), with class you can do different things, such as calling functions
class TextDataset(Dataset):  # Inherits from PyTorch's Dataset class
    def __init__(self, encodings, labels):
        self.input_ids = encodings['input_ids']       # Token IDs from tokenizer
        self.attention_mask = encodings['attention_mask']  # Mask for padding
        self.labels = torch.tensor(labels)  # Convert labels to tensors
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],       # Token IDs for one sample
            'attention_mask': self.attention_mask[idx],  # Mask for one sample
            'labels': self.labels[idx]              # Label for one sample
        }
    def __len__(self):
        return len(self.labels)  # Total number of samples

def get_encode_tokenize_data(path, model_path):
    logger.info("Loading data...")
    df = pd.read_csv(path)
    y = df["logical_fallacies"]
    X = df["text"]
    logger.info("Train test split, test-size 0.3")
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

    logging.info('encode the label column')
    le = LabelEncoder()
    y_train = le.fit_transform(y_train) 
    y_test = le.transform(y_test)
    
    logger.info('create tokenizer & load model')
    # tokenization after train test split to prevent data leakage
    #added use_fast=False to prevent tokenization error (might happen when using fast tokenization)
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

    train_encodings = tokenize(X_train.to_list())
    test_encodings = tokenize(X_test.to_list())
    train_dataset = TextDataset(train_encodings, y_train)
    test_dataset = TextDataset(test_encodings, y_test)

    return train_dataset, test_dataset, y_train


In [29]:
train_dataset, test_dataset, y_train = get_encode_tokenize_data(DATA_PATH, MODEL_PATH)

2025-04-04 11:19:36,638: Loading data...
2025-04-04 11:19:36,690: Train test split, test-size 0.3
2025-04-04 11:19:36,699: encode the label column
2025-04-04 11:19:36,702: create tokenizer & load model


In [45]:
y_train

array([5, 1, 1, ..., 3, 5, 4])

### done


In [ ]:
# df = pd.read_csv('../data/data_small.csv')

# Y = df["logical_fallacies"]
# X = df["text"]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, Y, test_size=0.30, random_state=42)

https://stackoverflow.com/questions/76868251/how-to-load-deberta-v3-properly

In [ ]:
# tokenization after train test split to prevent data leakage

#added use_fast=False to prevent tokenization error (might happen when using fast tokenization)
# tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=False)

In [ ]:
# def tokenize(texts):
#     return tokenizer(
#         texts,
#         padding="max_length", #ensures that all tokenized sequences are padded to the same length, padding adds special tokens to shorter sequeces so they match the maximum length
#         truncation=True, #if sequence exceeds max, it will be trucated
#         max_length=512, #for most transformer models, 512 is a common limit for maximum length
#         return_tensors="pt" #converts the output to pytorch tensors
#     )

In [ ]:
# train_encodings = tokenize(X_train.to_list())
# test_encodings = tokenize(X_test.to_list())

### Convert string labels to integers

In [ ]:
# le = LabelEncoder()
# y_train = le.fit_transform(y_train) 
# y_test = le.transform(y_test)

### Dataset Preparation for usage in model training

needed to create a PyTorch Dataset object that:
- organizes tokenized text
- pairs them with corresponding labels
- structures everything for batch processing during training

In [ ]:
# #object oriented programming (class is the object), with class you can do different things, such as calling functions

# class TextDataset(Dataset):  # Inherits from PyTorch's Dataset class
#     def __init__(self, encodings, labels):
#         self.input_ids = encodings['input_ids']       # Token IDs from tokenizer
#         self.attention_mask = encodings['attention_mask']  # Mask for padding
#         self.labels = torch.tensor(labels)  # Convert labels to tensors
#     def __getitem__(self, idx):
#         return {
#             'input_ids': self.input_ids[idx],       # Token IDs for one sample
#             'attention_mask': self.attention_mask[idx],  # Mask for one sample
#             'labels': self.labels[idx]              # Label for one sample
#         }
#     def __len__(self):
#         return len(self.labels)  # Total number of samples

In [ ]:
# train_dataset = TextDataset(train_encodings, y_train)
# test_dataset = TextDataset(test_encodings, y_test)

In [18]:
train_dataset[0]

{'input_ids': tensor([    1,   877,  1002,   732,  2740,   264, 23082, 23234,   270, 46439,
         18012,   392,  1033,   824,   260,     2,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

### Zero Shot Inference 

In [47]:
# # disable upper limit for memory
# os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# # Allows up to 100% of available memory
# torch.mps.set_per_process_memory_fraction(1.0)  

# torch.mps.empty_cache()  # Clears unused GPU memory

In [48]:
# # Load fresh copy of base model (not train on our data)
# num_classes = len(df["logical_fallacies"].unique())
# base_model = AutoModelForSequenceClassification.from_pretrained(
#     "microsoft/deberta-v3-small",
#     num_labels=num_classes,
#     problem_type="single_label_classification"
# )

In [49]:
# def predict(model, encodings, batch_size=8):
#     # Set the model to evaluation mode
#     model.eval()
    
#     # Use GPU
#     device = torch.device("mps")
#     model.to(device)
    
#     # Perform inference
#     probabilities = []
#     for i in range(0, len(encodings["input_ids"]), batch_size):
#         with torch.no_grad():
#             batch = {
#                 "input_ids": encodings["input_ids"][i:i+batch_size].to(device),
#                 "attention_mask": encodings["attention_mask"][i:i+batch_size].to(device)
#             }
#             outputs = model(**batch)
#             probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
#             probabilities.extend(probs)
            
#         # Clear GPU memory after each batch
#         torch.mps.empty_cache()
    
#     return np.array(probabilities)

In [50]:
# # Get predictions for test data
# base_probs = predict(base_model, test_encodings, batch_size=8)

In [51]:
# # Get highest probability indices
# predicted_indices = np.argmax(base_probs, axis=1)  

In [52]:
# from sklearn.metrics import classification_report

# # Generate classification report
# report = classification_report(y_test, predicted_indices, target_names=le.classes_)
# print(report)

Note: This deberta model is actually not designed for zero shot, there is one by MoritzLauer which can be used without requiring training on data. So training on data is actually necessary! The DeBERTa used here is meant for supervised learning. 
Another option is to use BART, facebook/bart-large-mnli model.

**Zero-Shot Learning** </span> is a concept, that a model when trained on enough unlabeled data (unsupervised learning) is able to generalize/ recognize at inference time even though the model was not trained on the inference data. This can be used in NLP, Images etc.

### Model Initialization

I had to change configuration of accelerate, as it might still be configured to fp16 (mixed precision)(doesn't work on Apple M1 Pro):
- type in bash accelerate confic
- this machine
- no distributed training
- do you want to run your training on CPU only, say No, as MAC Apple M1 Pro has GPU
- do you wish to optimize script with torch dynamo: say "No" if using an Apple M1 Pro with MPS backend
- do you want to use mixed prexision: NO

In [47]:
num_classes = len(np.unique(y_train))
num_classes

6

In [48]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_TRAINING_PATH,
    num_labels=num_classes,
    problem_type="single_label_classification"
)

model.gradient_checkpointing_enable()  # force model to use gradient checkpointing to save memory

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Class imbalance

In [49]:
OUTPUT_DIR = "../models/LLM_deberta_v3_small_class_imbalance/trainer_output"

In [ ]:
class WeightedLossTrainer(Trainer):
    def __init__(self, class_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.class_weights = class_weights
        if self.class_weights is not None:
            # Move weights to device after model initialization
            self._move_weights_to_device()
    
    def _move_weights_to_device(self):
        self.class_weights = self.class_weights.to(self.model.device)

    def compute_loss(
        self, 
        model, 
        inputs, 
        return_outputs=False, 
        num_items_in_batch=None  # Add this parameter
    ):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                       labels.view(-1))
        return (loss, outputs) if return_outputs else loss
        

def createTrainer(
    output_dir,
    y_train, 
    class_weight=False, 
    epochs=3, 
    learning_rate=5e-5, 
    weight_decay = 0, 
    train_batch_size = 4, 
    eval_batch_size=8
    ):
    logging.info('defining training arguments')
    training_args = TrainingArguments(
            output_dir=output_dir, # to sve results
            num_train_epochs=epochs,
            per_device_train_batch_size=train_batch_size, #small to save memory
            per_device_eval_batch_size=eval_batch_size, #small to save memory
            learning_rate=learning_rate, #standard for deberta; maybe try 6e-6
            weight_decay=weight_decay,
            eval_strategy="epoch",
            logging_steps=50,
            save_strategy="epoch",
            load_best_model_at_end=True
        )

    computed_class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    # Convert class weights to tensor
    computed_class_weights_tensor = torch.tensor(computed_class_weights, dtype=torch.float32)

    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        return {'accuracy': accuracy_score(p.label_ids, preds)}

    if class_weight==False:
        logging.info('get normal trainer')
        return Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                compute_metrics=compute_metrics
            )
    else:
        logging.info('get weighted loss trainer')
        return WeightedLossTrainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                compute_metrics=compute_metrics,
                class_weights=computed_class_weights_tensor
            )



In [ ]:
# epoch = 3
# learning_rate=2e-5 #standard for deberta; maybe try 6e-6
# weight_decay=0.01
# per_device_train_batch_size=4 #small to save memory
# per_device_eval_batch_size=8 

In [ ]:
trainer = createTrainer(class_weight=False, output_dir= OUTPUT_DIR, y_train=y_train, epochs=3, learning_rate=2e-5, weight_decay = 0.01, train_batch_size=4, eval_batch_size=8 )


2025-04-04 11:56:08,465: defining training arguments
2025-04-04 11:56:08,471: get normal trainer


In [ ]:
# Class-balanced trainer
# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

In [ ]:
# # Convert class weights to tensor
# class_weights = torch.tensor(class_weights, dtype=torch.float32)

In [ ]:

# class WeightedLossTrainer(Trainer):
#     def __init__(self, class_weights=None, **kwargs):
#         super().__init__(**kwargs)
#         self.class_weights = class_weights
#         if self.class_weights is not None:
#             # Move weights to device after model initialization
#             self._move_weights_to_device()
    
#     def _move_weights_to_device(self):
#         self.class_weights = self.class_weights.to(self.model.device)

#     def compute_loss(
#         self, 
#         model, 
#         inputs, 
#         return_outputs=False, 
#         num_items_in_batch=None  # Add this parameter
#     ):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits
#         loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
#                        labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

### Training Confguration 

In [ ]:
# Trainer with class weights

# training_args = TrainingArguments(
#     output_dir='../models/LLM_deberta_v3_small_class_imbalance/trainer_output', # to sve results
#     num_train_epochs=3,
#     per_device_train_batch_size=4, #small to save memory
#     per_device_eval_batch_size=8, #small to save memory
#     learning_rate=2e-5, #standard for deberta; maybe try 6e-6
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     logging_steps=50,
#     save_strategy="epoch",
#     load_best_model_at_end=True
# )

# def compute_metrics(p):
#     preds = p.predictions.argmax(-1)
#     return {'accuracy': accuracy_score(p.label_ids, preds)}

# trainer = WeightedLossTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics,
#     class_weights=class_weights
# )

In [ ]:
# # trainer without class weights

# training_args = TrainingArguments(
#     output_dir='../models/LLM_deberta_v3_small_class_imbalance/trainer_output', # to sve results
#     num_train_epochs=3,
#     per_device_train_batch_size=4, #small to save memory
#     per_device_eval_batch_size=8, #small to save memory
#     learning_rate=2e-5, #standard for deberta; maybe try 6e-6
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     logging_steps=50,
#     save_strategy="epoch",
#     load_best_model_at_end=True
# )

# def compute_metrics(p):
#     preds = p.predictions.argmax(-1)
#     return {'accuracy': accuracy_score(p.label_ids, preds)}

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics
# )

## Execute Training

In [59]:
torch.mps.empty_cache()  # Clears unused GPU memory

In [60]:
# disable upper limit for memory
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# Allows up to 100% of available memory
torch.mps.set_per_process_memory_fraction(1.0)  

In [61]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.991800,0.982793,0.641333
2,0.833200,1.239198,0.740000
3,0.658600,1.400739,0.751333


TrainOutput(global_step=2625, training_loss=0.9162530321393694, metrics={'train_runtime': 2004.204, 'train_samples_per_second': 5.239, 'train_steps_per_second': 1.31, 'total_flos': 1391056450560000.0, 'train_loss': 0.9162530321393694, 'epoch': 3.0})

In [64]:
results = trainer.evaluate()
print(f"Test Accuracy: {results['eval_accuracy']:.4f}")

Test Accuracy: 0.6413


In [65]:
output = trainer.predict(test_dataset)
predictions = np.argmax(output.predictions, axis=1)
y_true = output.label_ids

In [66]:
# Generate classification report
print(classification_report(y_true, predictions, target_names=le.classes_))

# Generate confusion matrix
cm = confusion_matrix(y_true, predictions)
print("Confusion Matrix:")
print(cm)

                       precision    recall  f1-score   support

           ad_hominem       0.62      0.68      0.65       159
  appeal_to_authority       0.42      0.74      0.54        97
    appeal_to_emotion       0.80      0.56      0.66       227
        false_dilemma       0.75      0.65      0.69       133
faulty_generalization       0.41      0.73      0.52       202
                 none       0.82      0.62      0.70       682

             accuracy                           0.64      1500
            macro avg       0.63      0.66      0.63      1500
         weighted avg       0.70      0.64      0.65      1500

Confusion Matrix:
[[108  10  15   0  10  16]
 [  6  72   3   2   6   8]
 [ 27   4 128   3  42  23]
 [  5   2   4  86  13  23]
 [ 11  12   4   3 147  25]
 [ 18  71   7  21 144 421]]


### Save model

I saved the model here as a SK model, although it is a Pytorch model. Keep that in mind!

Save model:
import mlflow.pytorch
mlflow.pytorch.save_model(model, "deberta_model")

Load model (correct way):
model = mlflow.pytorch.load_model("deberta_model")

In [68]:
#save with sklearn
path_sk = "../models/LLM_deberta_v3_small_class_imbalance/sk_learn_model"
save_model(sk_model=model, path=path_sk)

2025/04/03 14:03:26 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [70]:
#save with pytorch
path_pt = "../models/LLM_deberta_v3_small_class_imbalance/pytorch_model"
import mlflow.pytorch
mlflow.pytorch.save_model(model, path=path_pt)

### Load model

In [11]:
import mlflow.sklearn
path_sk = "../models/LLM_deberta_v3_small_class_imbalance/sk_learn_model"
model = mlflow.sklearn.load_model(path_sk)

In [13]:
import mlflow.pytorch
path_pt = "../models/LLM_deberta_v3_small_class_imbalance/pytorch_model"
model = mlflow.pytorch.load_model(path_pt)

### Make predictions based on reloaded model

Here the code works, although I loaded it as a Sklearn model, because I manually converted the logits to probabilites with torch.softmax. 

mlflow.sklearn.load_model() accidentally worked because MLflow can sometimes load PyTorch models as generic Python objects, but this isn't reliable

In [ ]:
# # batch can be changed both codes, now used the upper one
# # upper one makes it more generalized for dynamic inputs, as the lower one only handles input_ids and attention_mask
# batch = {
#     key: val[i:i+batch_size].to(device) 
#     for key, val in encodings.items()
# }

# batch = {
#                 "input_ids": encodings["input_ids"][i:i+batch_size].to(device),
#                 "attention_mask": encodings["attention_mask"][i:i+batch_size].to(device)
#             }

In [14]:
## Function for prediction

def predict(model, encodings, batch_size=8):
    # Set the model to evaluation mode
    model.eval()
    
    # Use GPU
    device = torch.device("mps")
    model.to(device)
    
    # Perform inference
    probabilities = []
    for i in range(0, len(encodings["input_ids"]), batch_size):
        with torch.no_grad():
            batch = {
                key: val[i:i+batch_size].to(device) 
                for key, val in encodings.items()
            }
            outputs = model(**batch)
            probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
            probabilities.extend(probs)
            
        # Clear GPU memory after each batch
        torch.mps.empty_cache()
    
    return np.array(probabilities)

In [15]:
#needed to reduce the batch size, otherwise I had an error
# Get predictions for test data
base_probs = predict(model, test_encodings, batch_size=2)

In [16]:
# Get highest probability indices
predicted_labels = np.argmax(base_probs, axis=1)

In [17]:
# Get second highest probability indices
second_predicted_labels = np.argsort(base_probs, axis=1)[:, -2]  

In [18]:
#probabilites of first predicted
predicted_label_probs = base_probs[np.arange(len(predicted_labels)), predicted_labels]

In [20]:
#probabilites of second predicted
second_predicted_label_probs = np.sort(base_probs, axis=1)[:, -2]  

In [21]:
#for backend 
result = {
    "predicted_labels": predicted_labels,
    "predicted_label_probs": predicted_label_probs,
    "second_predicted_labels": second_predicted_labels,
    "second_predicted_label_probs": second_predicted_label_probs
}

In [27]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, predicted_labels, target_names=le.classes_)
print(report)

# Generate confusion matrix
cm = confusion_matrix(y_test, predicted_labels)
print("Confusion Matrix:")
print(cm)

                       precision    recall  f1-score   support

           ad_hominem       0.62      0.68      0.65       159
  appeal_to_authority       0.42      0.74      0.54        97
    appeal_to_emotion       0.80      0.56      0.66       227
        false_dilemma       0.75      0.65      0.69       133
faulty_generalization       0.41      0.73      0.52       202
                 none       0.82      0.62      0.70       682

             accuracy                           0.64      1500
            macro avg       0.63      0.66      0.63      1500
         weighted avg       0.70      0.64      0.65      1500

Confusion Matrix:
[[108  10  15   0  10  16]
 [  6  72   3   2   6   8]
 [ 27   4 128   3  42  23]
 [  5   2   4  86  13  23]
 [ 11  12   4   3 147  25]
 [ 18  71   7  21 144 421]]


In [23]:
from sklearn.preprocessing import LabelBinarizer

# 1. One-hot encode the true labels (y_test)
lb = LabelBinarizer()
y_true_onehot = lb.fit_transform(y_test)  # Shape: (n_samples, n_classes)

# 2. Compute Brier score for multiclass
brier_score = np.mean(np.sum((base_probs - y_true_onehot) ** 2, axis=1))
print("Multiclass Brier score:", brier_score)

Multiclass Brier score: 0.49395134797722307
